In [1]:
pip install pandas openpyxl sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Read the Excel file into a DataFrame
df1 = pd.read_excel('SQL Test Dataset.xlsx')  # Adjust file path if needed

# Display the DataFrame
df1.head()  # To see the first few rows of the data


,Order ID,Order Priority,Customer ID,Customer Name,Product Category,Product Name,Region,Order Date,Quantity Ordered,Unit Price,Sales
0,88522,High,3,Bonnie Potter,Office Supplies,SANFORD Liquid Accent™ Tank-Style Highlighters,West,2015-01-07,4,3.252500,13.01
1,90193,Not Specified,5,Ronnie Proctor,Furniture,Global Troy™ Executive Leather Low-Back Tilter,West,2015-06-13,12,530.237500,6362.85
2,90192,Critical,11,Marcus Dunlap,Furniture,"DAX Two-Tone Rosewood/Black Document Frame, De...",East,2015-02-15,22,9.597727,211.15
3,86838,Medium,14,Gwendolyn F Tyson,Furniture,Howard Miller 12-3/4 Diameter Accuwave DS ™ Wa...,Central,2015-05-12,16,72.778125,1164.45
4,86838,Medium,14,Gwendolyn F Tyson,Office Supplies,Newell 321,Central,2015-05-12,7,3.175714,22.23


In [3]:
import pandas as pd

# Read the Excel file into a DataFrame
df2 = pd.read_excel('orders.xlsx')  # Adjust file path if needed

# Display the DataFrame
df2.head()  # To see the first few rows of the data


,Order ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned


In [4]:
import pandas as pd

# Read the Excel file into a DataFrame
df3 = pd.read_excel('users.xlsx')  # Adjust file path if needed

# Display the DataFrame
df3.head()  # To see the first few rows of the data


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [5]:
from sqlalchemy import create_engine

# Create an SQLite in-memory database engine
engine = create_engine('sqlite://', echo=False)

# Load each DataFrame into its own SQL table
df1.to_sql('orders', con=engine, index=False, if_exists='replace')
df2.to_sql('returns', con=engine, index=False, if_exists='replace')
df3.to_sql('users', con=engine, index=False, if_exists='replace')


4

In [6]:
# Query data from sql_table
sql_data = pd.read_sql_query('SELECT * FROM orders', con=engine)
sql_data.head()


,Order ID,Order Priority,Customer ID,Customer Name,Product Category,Product Name,Region,Order Date,Quantity Ordered,Unit Price,Sales
0,88522,High,3,Bonnie Potter,Office Supplies,SANFORD Liquid Accent™ Tank-Style Highlighters,West,2015-01-07 00:00:00.000000,4,3.252500,13.01
1,90193,Not Specified,5,Ronnie Proctor,Furniture,Global Troy™ Executive Leather Low-Back Tilter,West,2015-06-13 00:00:00.000000,12,530.237500,6362.85
2,90192,Critical,11,Marcus Dunlap,Furniture,"DAX Two-Tone Rosewood/Black Document Frame, De...",East,2015-02-15 00:00:00.000000,22,9.597727,211.15
3,86838,Medium,14,Gwendolyn F Tyson,Furniture,Howard Miller 12-3/4 Diameter Accuwave DS ™ Wa...,Central,2015-05-12 00:00:00.000000,16,72.778125,1164.45
4,86838,Medium,14,Gwendolyn F Tyson,Office Supplies,Newell 321,Central,2015-05-12 00:00:00.000000,7,3.175714,22.23


In [7]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT u.Manager,SUM(o.Sales) AS Total_Sales
FROM orders o
JOIN users u ON o.Region=u.Region
GROUP BY u.Manager
ORDER BY Total_Sales DESC
LIMIT 1
'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Manager,Total_Sales
0,Erin,34760.98


In [8]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT o."Product Name",SUM(o."Quantity Ordered") as Total_Quantity_Ordered,SUM(o.Sales) as Total_Sales
FROM orders o
GROUP BY o."Product Name"
ORDER BY Total_Sales DESC
LIMIT 1 OFFSET 1 
'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Product Name,Total_Quantity_Ordered,Total_Sales
0,"Global High-Back Leather Tilter, Burgundy",61,7119.38


In [9]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT o."Customer Name",COUNT(r."Order ID") as Total_Order_returned 
FROM orders o
JOIN returns r ON o."Order ID"=r."Order ID"
GROUP BY o."Customer Name"
ORDER BY Total_Order_returned DESC
LIMIT 3
'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Customer Name,Total_Order_returned


In [10]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT strftime('%Y-%m',o."Order Date") AS Month, 
SUM(o."Quantity Ordered") AS Total_Products_Sold, 
SUM(o.Sales) AS Total_Sales
FROM Orders o
GROUP BY strftime('%Y-%m',o."Order Date")
ORDER BY Month ASC;

'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Month,Total_Products_Sold,Total_Sales
0,2015-01,102,15584.08
1,2015-02,133,8864.52
2,2015-03,89,20676.77
3,2015-04,44,136.59
4,2015-05,305,6261.56


In [11]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT o."Product Name", 
SUM(o."Quantity Ordered") AS Total_Units_Sold, 
SUM(o.Sales) AS Total_Sales
FROM Orders o
GROUP BY o."Product Name"
ORDER BY Total_Units_Sold DESC
'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Product Name,Total_Units_Sold,Total_Sales
0,Hewlett Packard 6S Scientific Calculator,89,1146.08
1,DAX Natural Wood-Tone Poster Frame,87,2332.46
2,"Global High-Back Leather Tilter, Burgundy",61,7119.38
3,Advantus Plastic Paper Clips,58,293.06
4,Hon Valutask™ Swivel Chairs,43,4083.19


In [12]:
df1.replace(90193,740,inplace=True)
df1.head()

,Order ID,Order Priority,Customer ID,Customer Name,Product Category,Product Name,Region,Order Date,Quantity Ordered,Unit Price,Sales
0,88522,High,3,Bonnie Potter,Office Supplies,SANFORD Liquid Accent™ Tank-Style Highlighters,West,2015-01-07,4,3.252500,13.01
1,740,Not Specified,5,Ronnie Proctor,Furniture,Global Troy™ Executive Leather Low-Back Tilter,West,2015-06-13,12,530.237500,6362.85
2,90192,Critical,11,Marcus Dunlap,Furniture,"DAX Two-Tone Rosewood/Black Document Frame, De...",East,2015-02-15,22,9.597727,211.15
3,86838,Medium,14,Gwendolyn F Tyson,Furniture,Howard Miller 12-3/4 Diameter Accuwave DS ™ Wa...,Central,2015-05-12,16,72.778125,1164.45
4,86838,Medium,14,Gwendolyn F Tyson,Office Supplies,Newell 321,Central,2015-05-12,7,3.175714,22.23


In [13]:
df2.head(10)

,Order ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned
5,740,Returned
6,775,Returned
7,833,Returned
8,902,Returned
9,3300,Returned


In [14]:
# Perform a JOIN between order_table and users_table
query = '''
SELECT o."Customer Name",COUNT(r."Order ID") as Total_Order_returned 
FROM orders o
JOIN returns r ON o."Order ID"=r."Order ID"
GROUP BY o."Customer Name"
ORDER BY Total_Order_returned DESC
LIMIT 3
'''
joined_data = pd.read_sql_query(query, con=engine)

# Display the result of the JOIN
joined_data.head()

,Customer Name,Total_Order_returned
